## Import

In [1]:
import torch
import numpy as np
import pandas as pd

In [31]:
import urllib
url = "https://raw.githubusercontent.com/Jwizzed/ml-journey/main/TT.py"
urllib.request.urlretrieve(url, "TT.py")

In [3]:
!ls

McDonald's Store Reviews (DistilBert).ipynb

McDonald_s_Reviews.csv

TT.py

__pycache__

checkpoints


In [2]:
import TT

In [3]:
import huggingface_hub
huggingface_hub.login()

In [4]:
import torch

# Set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Create data and send it to the device
x = torch.rand(size=(3, 4)).to(device)
x

tensor([[0.6696, 0.7175, 0.7766, 0.4159],
        [0.2291, 0.8230, 0.3461, 0.7040],
        [0.3853, 0.0062, 0.7341, 0.9616]], device='mps:0')

## Get data

In [158]:
!kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



100%|██████████████████████████████████████| 1.78M/1.78M [00:01<00:00, 1.78MB/s]

100%|██████████████████████████████████████| 1.78M/1.78M [00:01<00:00, 1.57MB/s]


In [159]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

McDonald's Store Reviews.ipynb checkpoints

McDonald_s_Reviews.csv         mcdonalds-store-reviews.zip

TT.py                          test

__pycache__


In [160]:
TT.unzip("mcdonalds-store-reviews.zip", delete_original=True)

Unzipped Successfully


In [161]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

McDonald's Store Reviews.ipynb __pycache__

McDonald_s_Reviews.csv         checkpoints

TT.py                          test


In [5]:
orig_df = pd.read_csv("McDonald_s_Reviews.csv", encoding_errors="ignore", skiprows=lambda i: i % 50 != 0)
df = orig_df.copy()
df.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,50,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a year ago,"Normally this location does pretty good, but l...",1 star
1,100,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a year ago,Mcdonalds is great but they really need to hir...,4 stars
2,150,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",4 years ago,Always have had a positive experience. Arthur...,5 stars
3,200,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 years ago,Kiosk is a joke. Takes longer using it than ju...,2 stars
4,250,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 years ago,"They had 2 for 4$ McGriddles at the time, what...",5 stars


In [6]:
df.category.unique()

array(['Fast food restaurant'], dtype=object)

In [7]:
df.store_name.unique()

array(["McDonald's"], dtype=object)

In [8]:
df[["rating_count", "rating"]].tail()

,rating_count,rating
662,"2,810",5 stars
663,"2,810",2 stars
664,"2,810",1 star
665,"2,810",1 star
666,"2,810",1 star


In [9]:
df = df[["review", "rating"]]
df

,review,rating
0,"Normally this location does pretty good, but l...",1 star
1,Mcdonalds is great but they really need to hir...,4 stars
2,Always have had a positive experience. Arthur...,5 stars
3,Kiosk is a joke. Takes longer using it than ju...,2 stars
4,"They had 2 for 4$ McGriddles at the time, what...",5 stars
...,...,...
662,Awesome,5 stars
663,Poor,2 stars
664,"Very slow, too slow",1 star
665,Terrible!,1 star


## Data Preprocess

In [15]:
df.isna().sum(), df.shape

(review    0
 rating    0
 dtype: int64,
 (667, 2))

In [16]:
df.dropna(inplace=True)
df.shape

(667, 2)

In [17]:
df.drop_duplicates("review", keep="last", inplace=True)
df.rename(columns={"rating":"labels", "review":"text"}, inplace=True)
df.head()

,text,labels
0,"Normally this location does pretty good, but l...",1 star
1,Mcdonalds is great but they really need to hir...,4 stars
2,Always have had a positive experience. Arthur...,5 stars
3,Kiosk is a joke. Takes longer using it than ju...,2 stars
4,"They had 2 for 4$ McGriddles at the time, what...",5 stars


In [18]:
df.labels.unique()

array(['1 star', '4 stars', '5 stars', '2 stars', '3 stars'], dtype=object)

In [19]:
df["labels"] = df["labels"].replace({"1 star":0, "2 stars": 1, "3 stars": 2, "4 stars": 3, "5 stars": 4 })
df = df.loc[df['text'].str.contains(r'[^\x00-\x7F]+') == False]
df.head()

,text,labels
0,"Normally this location does pretty good, but l...",0
1,Mcdonalds is great but they really need to hir...,3
2,Always have had a positive experience. Arthur...,4
5,Fries tasted like they were cooked in grease f...,0
6,"Good food, especially late",3


In [20]:
import datasets

In [21]:
max_length = 128 
batch_size = 32 

dataset = datasets.Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset = dataset.remove_columns(["__index_level_0__"])
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 412
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 103
    })
})

In [22]:
id2label = {0: "1 star", 1: "2 stars", 2: "3 stars", 3: "4 stars", 4: "2 stars"}
label2id = {"1 star":0, "2 stars":1, "3 stars":2, "4 stars":3, "5 stars":4}

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,TrainingArguments, Trainer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=5, id2label=id2label, label2id=label2id).to(device)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']

- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_cla

In [24]:
avg_length = int(np.mean([len(sentence) for sentence in df.text]))
avg_length

140

In [25]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"],padding=True, truncation=True, return_tensors="pt").to(device)
        
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",  
        max_length=avg_length,  
        truncation=True, 
        return_tensors="pt"
    ).to(device)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/412 [00:00<?, ? examples/s]

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

## Model

In [26]:
data_collator = DataCollatorWithPadding(tokenizer)

In [27]:
def compute_metrics(eval_pred):
    acc_metric = datasets.load_metric('accuracy')
    f1_metric = datasets.load_metric('f1')
    y_logits, y_labels = eval_pred
    y_pred = np.argmax(y_logits, axis=-1)
    acc = acc_metric.compute(predictions=y_pred, references=y_labels)['accuracy']
    f1 = f1_metric.compute(predictions=y_pred, references=y_labels, average='macro')['f1']
    
    return {'accuracy':acc, 'f1':f1}

In [28]:

training_args = TrainingArguments(
    output_dir="checkpoints",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,

)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/Users/krittinsetdhavanich/pytorch-test/touch-env/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning

  warnings.warn(

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.486685,0.504854,0.257394
2,No log,1.289004,0.553398,0.280882
3,No log,1.204338,0.563107,0.285965
4,No log,1.176864,0.563107,0.285965
5,No log,1.161213,0.563107,0.285965


/var/folders/54/bsd8kfld1_xcw8npb7rc2qgw0000gn/T/ipykernel_60090/1295628286.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate

  acc_metric = datasets.load_metric('accuracy')


TrainOutput(global_step=130, training_loss=1.2232698880709134, metrics={'train_runtime': 376.6287, 'train_samples_per_second': 5.47, 'train_steps_per_second': 0.345, 'total_flos': 74620393932000.0, 'train_loss': 1.2232698880709134, 'epoch': 5.0})

In [30]:
trainer.evaluate()

{'eval_loss': 1.1612132787704468,
 'eval_accuracy': 0.5631067961165048,
 'eval_f1': 0.28596491228070176,
 'eval_runtime': 6.9661,
 'eval_samples_per_second': 14.786,
 'eval_steps_per_second': 1.005,
 'epoch': 5.0}